# Laboratorio Regresión Lineal


In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score


from sklearn.preprocessing import StandardScaler

from tqdm import tqdm


# Ignorar los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')

pd.options.display.float_format = "{:,.2f}".format

import sys
sys.path.append("../../")
from src.soporte_metricas import (
    obtener_metricas
)

# Vamos a realizar Cross Validation
Usando el encoding Target
- El objetivo es ver si nuestro modelo puede mejorar al realizar varias iteraciones. de la regresión Lineal.

In [2]:
df = pd.read_csv("../../datos/04_Modelo4/06_autorenew_target_encoded.csv",index_col=0)
df.sample()

,offerType,brand,vehicleType,fuelType,gearbox,notRepairedDamage,kilometer,price,yearOfRegistration,powerCV_robust_scaler
124868,"6,065.12","2,990.11","2,840.04","5,152.20","4,824.16","7,212.17","8,664.56",950.00,"1,910.74","1,631.88"


In [3]:
X = df.drop(columns="price")
y = df[["price"]]
X_train, X_test, y_train, y_test = train_test_split(X, # Todo menos la Variable Respuesta
                                                    y, # La variable Respuesta
                                                    train_size=0.7, # Como dividir los datos en este caso 80% train, 20% test
                                                    random_state=42, # Semilla, para que lo haga igual para cualquier persona
                                                    shuffle=True 
                                                    )


# Hagamos el modelo Linear y Recapitulemos

In [4]:
modelo_lineal = LinearRegression()
modelo_lineal.fit(X_train, y_train)
y_pred_test = modelo_lineal.predict(X_test)
y_pred_train = modelo_lineal.predict(X_train)

# Veamos sus métricas

In [5]:
obtener_metricas(y_train,y_pred_train,y_test,y_pred_test)

,r2_score,MAE,MSE,RMSE
train,0.24,"3,134.71","166,044,677.48","12,885.83"
test,0.33,"3,113.28","98,421,486.38","9,920.76"


### Como vimos anteriormente:
- No es viable el error que tenemos ni el R2 es aceptable
- Para solucionarlo, vamos a intentar realizar K-Fold Cross Validation
### ¿Porque?
- Una de las principales causas es que tenemos un conjunto de más de 100.000 datos, lo que lo hace un conjunto moderado de datos, por rendimiento y tiempo empezaremos por este

In [6]:
df.shape

(356472, 10)

### ¿Que puedo meter en scoring?

- r2
- MAE: neg_mean_absolute_error
- MSE: neg_mean_squared_error
- RMSE: neg_root_mean_squared_error



# Empecemos calculando el R2
- En varias iteraciones veremos si mejora potencialmente
### ¿Cuántos splits?
- Voy a probar con 5, 50, 100, 500 y 1000, de ahí vemos la media de r2


In [7]:
splits = [5,50,100,500,1000]
for split in tqdm(splits):
    kf = KFold(n_splits=split,
            shuffle=True,
            random_state=42)
    score_r2 = cross_val_score(modelo_lineal,
                            X,
                            y,
                            cv=kf,
                            scoring="r2")
    print(f"r2 en {split} splits: {np.mean(score_r2)}")

 20%|██        | 1/5 [00:00<00:01,  2.41it/s]

r2 en 5 splits: 0.27783271058462944


 40%|████      | 2/5 [00:04<00:08,  2.80s/it]

r2 en 50 splits: 0.3973450216173445


 60%|██████    | 3/5 [00:14<00:11,  5.80s/it]

r2 en 100 splits: 0.4486929943634437


 80%|████████  | 4/5 [01:04<00:23, 23.40s/it]

r2 en 500 splits: 0.537109177092594


100%|██████████| 5/5 [02:38<00:00, 31.80s/it]

r2 en 1000 splits: 0.5594964903539597


Tarda 2 mins
- r2 en 5 splits: 0.33618296668739367
- r2 en 50 splits: 0.41857590248457505
- r2 en 100 splits: 0.44196576819632055
- r2 en 500 splits: 0.5091593582512305
- r2 en 1000 splits: 0.5178094658590221

Antes de tener decision, veamos el error RMSE y su evolución

In [8]:
splits = [5,50,100,500,1000]
for split in tqdm(splits):
    kf = KFold(n_splits=split,
            shuffle=True,
            random_state=42)
    score_rmse = cross_val_score(modelo_lineal,
                            X,
                            y,
                            cv=kf,
                            scoring="neg_root_mean_squared_error")
    print(f"Error RMSE en {split} splits: {np.mean(score_rmse)}")

 20%|██        | 1/5 [00:00<00:02,  1.99it/s]

Error RMSE en 5 splits: -11866.264578887092


 40%|████      | 2/5 [00:05<00:08,  3.00s/it]

Error RMSE en 50 splits: -10233.310652850741


 60%|██████    | 3/5 [00:14<00:11,  5.86s/it]

Error RMSE en 100 splits: -9472.958930290763


 80%|████████  | 4/5 [01:00<00:21, 21.76s/it]

Error RMSE en 500 splits: -7789.712426972162


100%|██████████| 5/5 [02:34<00:00, 30.82s/it]

Error RMSE en 1000 splits: -7129.749957170835


Tarda 4 mins
- Error RMSE en 5 splits: -12267.559531024217
- Error RMSE en 50 splits: -10612.957198951419
- Error RMSE en 100 splits: -10273.086514930843
- Error RMSE en 500 splits: -8429.755759419888
- Error RMSE en 1000 splits: -7765.457287420199

# La Realidad
- Aún generando grupos de 1000 con los datos, no somos capaces de reducir el error de forma sustancial
- Si es verdad que el R2 crece a más grupos, y lo más probable es que siguiera creciendo
- Por eso vemos el error RMSE, para saber si nos reduce el error
- Y vemos que aún reduciendo el error, una media de 7765€ de error el el precio sigue siendo inaceptable

# ¿Entonces?
- Vamos a realizar un Decision Tree en el jupyter 07